In [1]:
import nltk
import torch
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
from peft import PeftConfig, PeftModel
from t5.dataset import load_spider_datasets
from t5.inference import inference, evaluate_result

/home/roxyrong/anaconda3/envs/xlang/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(torch.cuda.is_available())

In [ ]:
nltk.download('punkt')

In [ ]:
# parameters

tokenizer_name = 't5-base'
base_model_name = "RoxyRong/t5_base_finetuned_test_10"
peft_model_name = "RoxyRong/t5_base_soft_prompt_vocab_sampling"
result_path = f'results/predicted_result_t5_base_finetuned_10.txt'

eval_soft_prompt = False

In [ ]:
# evaluate 

tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

if eval_soft_prompt:
    peft_model_id = peft_model_name
    config = PeftConfig.from_pretrained(peft_model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained("RoxyRong/t5_base_finetuned_test_5")
    model = PeftModel.from_pretrained(model, peft_model_id)

model = model.to("cuda")

In [ ]:
# load dataset
_, _, dev_spider = load_spider_datasets()

In [ ]:
# inference
inference(dev_spider, model, tokenizer, result_path)

In [ ]:
# evaluate 
evaluate_result(result_path)